# Environment Preparation.

In [265]:
import pandas as pd
import time
import tracemalloc

# Reading CSV File.

In [266]:
df = pd.read_csv('rentaldb_ca_cleaned.csv')

# Reading head using Pandas.

In [267]:
print(df.head())


   id    price  square_feet  bedrooms  bathrooms   cityname province  \
0   1  1451.01       1145.0         2        1.6     Ottawa       ON   
1   2  1856.45       1076.0         2        1.0   Edmonton       AB   
2   3  1345.59        353.0         2        2.8  Vancouver       BC   
3   4  3409.37        268.0         1        2.0  Vancouver       BC   
4   5  3940.95        567.0         1        1.4   Montreal       QC   

    latitude   longitude                        amenities posted_date  \
0  45.420210  -75.711618                      Gym;Laundry   9/25/2025   
1  53.479743 -113.493715            Elevator;Laundry;Pool  11/17/2025   
2  49.307885 -123.097556  Laundry;Storage;Parking;Balcony  11/23/2025   
3  49.287966 -123.062954     Gym;Storage;Elevator;Parking    7/8/2025   
4  45.507787  -73.625302                     Pool;Laundry   6/11/2025   

    building  postal_code neighbourhood  pets_allowed included_utilities  
0  Townhouse        89211          land          True

# Reading Columns using Pandas.

In [268]:
print(df.columns)

Index(['id', 'price', 'square_feet', 'bedrooms', 'bathrooms', 'cityname',
       'province', 'latitude', 'longitude', 'amenities', 'posted_date',
       'building', 'postal_code', 'neighbourhood', 'pets_allowed',
       'included_utilities'],
      dtype='object')


# Linear Search for Price.

In [269]:
# Linear search by price
def linear_search_price(df, target_price):
    return df.index[df['price'] == target_price].tolist()

In [270]:
target_price = 1826.54
print("Linear search price $1800:", linear_search_price(df, target_price))

Linear search price $1800: [49]


# Linear Search for City.

In [271]:
# Linear search by city
def linear_search_city(df, target_city):
    return df.index[df['cityname'] == target_city].tolist()


In [272]:
target_city = 'Toronto'
print("Linear search city Toronto:", linear_search_city(df, target_city))

Linear search city Toronto: [14, 16, 17, 31, 38, 40, 46, 52, 54, 55, 65, 73, 74, 76, 103, 107, 132, 141, 145, 148, 150, 157, 176, 185, 189, 201, 204, 207, 209, 223, 243, 246]


# Binary Search for Price.

In [273]:
df['orig_index'] = df.index

In [274]:
df_sorted = df.sort_values('price').reset_index(drop=True)

In [275]:
def binary_search_price(df, target_price, tol=1e-2):
    low = 0
    high = len(df) - 1
    results = []

    while low <= high:
        mid = (low + high) // 2
        mid_value = df.loc[mid, 'price']

        if abs(mid_value - target_price) < tol:
            # collect all duplicates
            i = mid
            while i >= 0 and abs(df.loc[i, 'price'] - target_price) < tol:
                results.append(df.loc[i, 'orig_index'])
                i -= 1

            i = mid + 1
            while i < len(df) and abs(df.loc[i, 'price'] - target_price) < tol:
                results.append(df.loc[i, 'orig_index'])
                i += 1

            return sorted(results)

        elif mid_value < target_price:
            low = mid + 1
        else:
            high = mid - 1

    return []  # not found

In [276]:
price_target = 1826.54
binary_matches = binary_search_price(df_sorted, price_target)

if binary_matches:
    result = int(binary_matches[0])
else:
    result = None

print("Binary search result (row index):", result)


Binary search result (row index): 49


# Binary Search for City.

In [277]:
df['orig_index'] = df.index

In [278]:
df_sorted_city = df.sort_values('cityname').reset_index(drop=True)

In [279]:
def binary_search_city(df, target_city):
    """
    Returns a list of original row indices where cityname == target_city
    """
    low = 0
    high = len(df) - 1
    results = []

    while low <= high:
        mid = (low + high) // 2
        mid_value = df.loc[mid, 'cityname']

        if mid_value == target_city:
            # collect all duplicates
            i = mid
            while i >= 0 and df.loc[i, 'cityname'] == target_city:
                results.append(df.loc[i, 'orig_index'])
                i -= 1

            i = mid + 1
            while i < len(df) and df.loc[i, 'cityname'] == target_city:
                results.append(df.loc[i, 'orig_index'])
                i += 1

            return sorted(results)  # return sorted indices

        elif mid_value < target_city:
            low = mid + 1
        else:
            high = mid - 1

    return []  # not found

In [280]:
target_city = "Toronto"
binary_matches = binary_search_city(df_sorted_city, target_city)

if binary_matches:
    clean_indices = [int(i) for i in binary_matches]
    print("All matching row indices:", clean_indices)
else:
    print("No matches found")


All matching row indices: [14, 16, 17, 31, 38, 40, 46, 52, 54, 55, 65, 73, 74, 76, 103, 107, 132, 141, 145, 148, 150, 157, 176, 185, 189, 201, 204, 207, 209, 223, 243, 246]


# Bubble Sort (Price)

In [281]:
def bubble_sort_by_price(df):
    # Making a copy so original DataFrame is not modified
    df_sorted = df.copy()
    n = len(df_sorted)

    # Bubble Sort algorithm
    for i in range(n):
        for j in range(0, n - i - 1):
            if df_sorted.loc[j, "price"] > df_sorted.loc[j + 1, "price"]:
                # Swap entire rows
                df_sorted.iloc[[j, j + 1]] = df_sorted.iloc[[j + 1, j]].values
    return df_sorted


In [282]:
#Apply the Bubble Sort
sorted_df = bubble_sort_by_price(df)
print(sorted_df.head(10))

    id     price  square_feet  bedrooms  bathrooms     cityname province  \
0   92   716.190        852.0         1        1.0  Quebec City       QC   
1   80   810.660        517.0         1        1.9      Calgary       AB   
2   31  1001.555        497.0         2        1.3     Montreal       QC   
3  102  1052.080        404.0         2        1.6  Mississauga       ON   
4   20  1086.960        714.0         1        1.5       Ottawa       ON   
5  155  1100.730        621.0         1        2.2  Mississauga       ON   
6  189  1150.860        405.0         3        2.1     Montreal       QC   
7  111  1171.810        687.0         5        2.3  Mississauga       ON   
8   47  1176.330        990.0         1        1.3      Toronto       ON   
9   71  1189.810        389.0         1        1.5  Mississauga       ON   

    latitude   longitude                                 amenities  \
0  46.805797  -71.205096  Elevator;Laundry;Storage;Parking;Balcony   
1  51.003304 -114.08862

# Insertion Sort (Price)

In [283]:
def insertion_sort_by_price(df):
    # Making a copy so the original DataFrame is not modified
    df_sorted = df.copy()
    n = len(df_sorted)

    # Insertion Sort algorithm
    for i in range(1, n):
        # Store the current row
        key_row = df_sorted.iloc[i].copy()
        j = i - 1

        # Move rows greater than key_row["price"] one step ahead
        while j >= 0 and df_sorted.loc[j, "price"] > key_row["price"]:
            df_sorted.iloc[j + 1] = df_sorted.iloc[j].values
            j -= 1

        # Place key_row in the correct position
        df_sorted.iloc[j + 1] = key_row.values

    return df_sorted




In [284]:
#Apply insertion sort
sorted_df = insertion_sort_by_price(df)
print(sorted_df.head())

    id     price  square_feet  bedrooms  bathrooms     cityname province  \
0   92   716.190        852.0         1        1.0  Quebec City       QC   
1   80   810.660        517.0         1        1.9      Calgary       AB   
2   31  1001.555        497.0         2        1.3     Montreal       QC   
3  102  1052.080        404.0         2        1.6  Mississauga       ON   
4   20  1086.960        714.0         1        1.5       Ottawa       ON   

    latitude   longitude                                 amenities  \
0  46.805797  -71.205096  Elevator;Laundry;Storage;Parking;Balcony   
1  51.003304 -114.088621                      Pet Friendly;Storage   
2  45.521779  -73.598714                          Elevator;Balcony   
3  43.581535  -79.627034                  Elevator;Balcony;Laundry   
4  45.381612  -75.669001                               Gym;Laundry   

  posted_date   building  postal_code neighbourhood  pets_allowed  \
0   9/10/2025      House        79501          side  

# Bubble Sort Timing

In [ ]:
start = time.perf_counter()
bubble_sorted = bubble_sort_by_price(df)
end = time.perf_counter()
t_bubble = end - start
print(f"Bubble Sort time: {t_bubble:.6f} seconds")

# Insertion Sort Timing

In [ ]:
start = time.perf_counter()
insertion_sorted = insertion_sort_by_price(df)
end = time.perf_counter()
t_insertion = end - start
print(f"Insertion Sort time: {t_insertion:.6f} seconds")


# Python Built-in sort_values

In [ ]:
start = time.perf_counter()
builtin_sorted = df.sort_values("price", ascending=True)
end = time.perf_counter()
t_builtin = end - start
print(f"Python built-in sort time: {t_builtin:.6f} seconds")


In [ ]:
# ----------- PRINT RESULTS -----------
print("====== TIME COMPARISON (seconds) ======")
print(f"Bubble Sort:      {t_bubble:.6f}")
print(f"Insertion Sort:   {t_insertion:.6f}")
print(f"Python sorted():  {t_python:.6f}")

# Measure Memory for Bubble Sort

In [ ]:
tracemalloc.start()  # start tracking memory

bubble_sorted = bubble_sort_by_price(df)

# Get memory stats
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()  # stop tracking

print(f"Bubble Sort peak memory: {peak} bytes")


# Measure Memory for Insertion Sort

In [ ]:
tracemalloc.start()

insertion_sorted = insertion_sort_by_price(df)

current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Insertion Sort peak memory: {peak} bytes")


# Measure Memory for Python Built-in sort_values

In [ ]:
tracemalloc.start()

builtin_sorted = df.sort_values("price", ascending=True)

current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Python built-in sort peak memory: {peak} bytes")


# Report wall-clock time and a brief interpretation (complexity, constant factors, data distribution effects)

Bubble Sort: slowest due to O(n²) complexity, Insertion Sort is faster for small or nearly sorted data, and Python’s built-in sort_values() is fastest thanks to optimized O(n log n) Timsort..

# Briefly interpret memory results: explain why certain algorithms allocate more and how data types/object overhead in pandas vs pure Python lists affect usage.

 Bubble Sort and Insertion Sort use slightly less memory since they sort in place, while Python’s sorted() uses more due to creating new lists and internal structures. Overhead from data types and object storage in Python also increases memory usage compared to simple in-place operations.




# What surprised you? When would your custom sort ever be preferable?

What surprised me was how fast the built-in sort is even on small datasets. Custom sorts are mainly useful for learning, teaching, or very small datasets where understanding the sorting mechanics matters more than performance.